Assumptions: Using an Ubuntu machine with sudo access. Have the latest Python. 
** LINUX NOTE ** You have to edit `bandgapReferenceCircuit/xschemrc` to replace my paths for skywater130 data and xschem libraries with your paths.
1. Install `ngspice`, `xschem`, open_pdks, and the skywater 130nm pdk
2. clone this repository: (my repository)
3. run `characterize.py`
    e.g.: `python3 characterize.py ngspice --modelFilePath <your paths>/skywater-pdk/libraries/sky130_fd_pr/latest/models/sky130.lib.spice`
4. run bipolar characterization spice file
    a. Open schematics/bipolar_charac...
    b. Netlist and Simulate the schemtaic. Make sure you changed `xschemrc` 
    `ngspice `
4. design bandgap circuit, including the amplifier, current mirrors, etc. (note: gm/Id based design can be used with lookupMos.py)

1. Bandgap Design
Read this paper: 
H. Banba et al., "A CMOS bandgap reference circuit with sub-1-V operation," in IEEE Journal of Solid-State Circuits, vol. 34, no. 5, pp. 670-674, May 1999, doi: 10.1109/4.760378.
https://ieeexplore.ieee.org/document/760378

2. Remaining Circuit Design
a. The amplifier is the source of loop gain in a feedback control loop. Check outo the small signal form to see negative feedback.
*Thought Provoking Question:* The amplifier is differential so there are two loops, negative and positive, from input to output. Why does one loop need a capacitor on it? How would removing the capacitor affect stability?
b. The amplifier is self-biased. You should work out, with the square law model and extracted model parameters, what is the dc voltage of the amplifier output? This node is the same as the current mirror input, which feeds into another current mirror. What is the relationship between the output dc voltage from an input dc common mode voltage? The range of that function impacts headroom of the amplifier.
c. The circuit requires start-up circuitry for *reliable* startup. Think about how this is a primitive digital control bit. 

My design can be understood through particular attention to tsmc_bandgap_real.sch. You need xschem with the xschem skywater-130 library to really read this file.

The tools I used for design are 
pyMOSchar/characterize.py to characterize NFET and PFET devices,
pyMOSchar/lookupMOS.py to use look-up tables of small signal parameters for many arrayed values of VGS, VDS, VSB, W, L. Another file, pyMOSchar/lookupExamples.py, may also help.

That's it. I hope you have fun! The rest of the repository is related to the bandgap I personally taped out during Stanford's Tapeout Series (https://priyanka-raina.github.io/teaching/)

Interact with some NFET and PFET data I generated. 
The locaiton of the raw data is:.
Read the code below this cell and run it. This data only has data for pairs of (width, length) = {(1, 0.9), (2, 1), (5, 1.1)} for nmos and (width, length) = {(2, 0.9), (4, 1), (10, 1.1)} for pmos.

In [ ]:
import lookupMOS as lk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

filename = ''
lk.init(filename)
VGS = np.linspace(0, 1.8, endpoint=True)
matplotlib.rc('xtick',labelsize=15)
matplotlib.rc('ytick',labelsize=15)
# print('Available Lenghts: {}'.format(lk.mosDat['nfet']['length']))
lengths = [0.35, 0.705102, 0.9250000] # micron
width = 1 # micron
types = ['pfet', 'nfet']
for typ in types:
    id = lk.lookup(typ, 'id', l=[l*1e6 for l in lengths], vds=1.8 ,vsb=0, vgs=VGS)
    gm = lk.lookup(typ, 'gm', l=[l*1e6 for l in lengths], vds=1.8,vsb=0, vgs=VGS)
    vt = lk.lookup(typ, 'vt', l=[l*1e6 for l in lengths], vds=1.8,vsb=0, vgs=VGS)
    fT = lk.lookup(typ, 'gm/cgg', l=[l*1e6 for l in lengths], vds=1.8, vgs=VGS)/2/np.pi
    K = [id[i]/(VGS-vt[i])**2 * lengths[i] / width for i in range(len(lengths))] # = 1/2 * mu_n * C_ox
    K_norm = [K[i] / (lengths[i] / width) for i in range(len(lengths))]
    figK, axK = plt.subplots()
    axK.set_xlabel('Vov (V)')
    axK.set_ylabel('log10 (1/2 * mu_n * C_ox) / (L / W)')
    axK.set_title('{} Normalized square Law K term'.format(typ))
    indc = (VGS-vt) > 0.
    for i, k in enumerate(K_norm):
        axK.plot(VGS[indc[i]]-vt[i][indc[i]], np.log10(k[indc[i]]), label=lengths[i])
    axK.legend()
    fig, ax1 = plt.subplots(3)

    ax1[0].set_xlabel('Vov (V)')
    ax1[0].set_ylabel('gm/Id (S/A)')
    ax1[0].set_title('{} Tradeoff: gm/Id vs. fT.'.format(typ))
    lns1 = []
    lns2 = []
    gmonid = [gm[j]/id[j] for j in range(len(lengths))]
    fom = [gmonid[i]*fT[i]/1e9 for i in range(len(lengths))]
    for i, gmid in enumerate(gmonid):
        lns1.append(ax1[0].plot(VGS-vt[i], gmid, 'o--', label=lengths[i]))
    ax1[0].legend()
    ax2 = ax1[0].twinx()
    ax2.set_ylabel('fT (GHz)')

    for i, f in enumerate(fT):
        lns2.append(ax2.plot(VGS-vt[i], f/1e9, '+--', label='fT'))

    ax1[0].grid(True)
    fig.tight_layout()
    for i, gi in enumerate(gmonid):
        ax1[1].plot(gi, gi*fT[i]/1e9, 'o--', label=lengths[i])
    ax1[1].legend()
    for i, gi in enumerate(gmonid):
        ax1[2].plot(VGS-vt[i], gi*fT[i]/1e9, 'o--', label=lengths[i])
    ax1[2].legend()

    ax1[1].set_xlabel('gm/Id (S/A)')
    ax1[1].set_ylabel('gm/Id*fT (S/A * GHz)')
    ax1[1].set_title('{} Figure of Merit.'.format(typ))
    ax1[1].grid(True)
    ax1[2].set_xlabel('Vov (V)')
    ax1[2].set_ylabel('gm/Id*fT (S/A * GHz)')
    ax1[2].set_title('{} Figure of Merit.'.format(typ))
    ax1[2].grid(True)
    fig.suptitle(filename + ' W = 1um')
    fig, ax = plt.subplots()
    ax.set_xlabel('Vgs (V)')
    ax.set_ylabel('I (uA)')
    ax.set_title('{} Id vs. Vgs. W = 1um'.format(typ))
    idarr = lk.lookup(typ, 'id', l=[l*1e6 for l in lengths], vds=1.8, vsb=0, vgs=VGS)
    for i,id in enumerate(idarr) :
        ax.plot(VGS, id*1e6, label=str((0.5 * i + 1)/1e6) + ' um')
    ax.legend()
plt.show()